In [1]:
import os
import sys
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
from stargazer.stargazer import Stargazer
from IPython.display import display, HTML
from sklearn.datasets import load_iris
from pystout import pystout
import matplotlib.pyplot as plt

import numpy as np

In [2]:
# Global Settings
pd.set_option("display.float_format", lambda x: "%.4f" % x)

In [3]:
cwd = os.getcwd()
# Find and import config file
config_path = os.getcwd()

sys.path.append(config_path)
import config

database = config.database
central_banks = config.central_banks
training_data = os.path.join(database, "Training Data")
fed_docs = config.fed_docs
ecb_docs = config.ecb_docs
boe_docs = config.boe_docs

sentiment = pd.date_range(start="1/1/1990", end="1/1/2024", freq="D")
sentiment = pd.DataFrame(sentiment, columns=["date"])

In [4]:
sentiment = pd.date_range(start="1/1/1990", end="12/31/2024", freq="D")
sentiment = pd.DataFrame(sentiment, columns=["date"])

In [5]:
# # HP Filter

url_map = pd.read_csv(os.path.join(cwd, "url_map.csv"))

for i in range(len(url_map)):
    temp_dates = pd.date_range(start="1/1/1990", end="12/31/2024", freq="D")
    temp_dates = pd.DataFrame(temp_dates, columns=["date"])
    sent = pd.read_csv(url_map["finbert_url"][i])
    sent = sent[["date", "sentiment"]]

    sent["date"] = pd.to_datetime(sent["date"])
    sent = sent.groupby("date").mean().reset_index()

    sent = pd.merge(temp_dates, sent, how="left", left_on="date", right_on="date")

    # Set 'date' as the index for resampling
    sent.set_index("date", inplace=True)

    # Drop NaNs
    sent = sent.dropna(subset=["sentiment"])

    # Reset index to turn 'date' back into a column
    sent.reset_index(inplace=True)

    filter_df = sent.copy(deep=True)

    cycle, trend = sm.tsa.filters.hpfilter(
        filter_df["sentiment"], 1600 * ((8 / 4) ** 4)
    )

    filter_df["sentiment_cycle"] = cycle
    filter_df["sentiment_trend"] = trend

    filter_df = filter_df[["date", "sentiment_cycle"]]
    sent = sent.drop(columns=["sentiment"])
    filter_df = filter_df.rename(columns={"sentiment_cycle": "sentiment"})
    sent = pd.merge(sent, filter_df, on="date", how="outer")
    sent = sent.groupby("date").mean().reset_index()

    sent = sent.rename(columns={"sentiment": url_map["document"][i]})
    sentiment = pd.merge(sentiment, sent, how="outer", left_on="date", right_on="date")

In [6]:
market = pd.read_csv(f"{database}/Market Data/All Market Data.csv")
market = market.rename(columns={"PCE": "pce", "GDP": "gdp", "GDPPOT": "gdp_pot"})
market["date"] = pd.to_datetime(market["date"])

sentiment = pd.merge(sentiment, market, how="left", left_on="date", right_on="date")

In [7]:
fed_sentiment = sentiment.copy(deep=True)
fed_sentiment = fed_sentiment[["date", "fed_minutes"]]
fed_sentiment = fed_sentiment.dropna(subset=["fed_minutes"])

fed_sentiment["count"] = range(len(fed_sentiment))
fed_sentiment = fed_sentiment[["date", "count"]]
sentiment = pd.merge(
    sentiment, fed_sentiment, how="left", left_on="date", right_on="date"
)
sentiment["count"] = sentiment["count"].ffill()

fed_sentiment = sentiment.copy(deep=True)
fed_sentiment = fed_sentiment[["count", "sp500_return"]]
fed_sentiment["log_returns"] = np.log(fed_sentiment["sp500_return"] + 1)
fed_sentiment = fed_sentiment[["count", "log_returns"]]
fed_sentiment = fed_sentiment.groupby("count").sum().reset_index()
fed_sentiment = fed_sentiment.groupby(fed_sentiment["count"]).mean().reset_index()

fed_temp = sentiment.copy(deep=True)
fed_temp = fed_temp[["count", "fed_minutes"]]
fed_temp = fed_temp.dropna(subset=["fed_minutes"])

fed_sentiment = pd.merge(
    fed_sentiment, fed_temp, how="left", left_on="count", right_on="count"
)

In [8]:
ecb_sentiment = sentiment.copy(deep=True)
ecb_sentiment = ecb_sentiment[["date", "press_conferences"]]
ecb_sentiment = ecb_sentiment.dropna(subset=["press_conferences"])

ecb_sentiment["ecb_count"] = range(len(ecb_sentiment))
ecb_sentiment = ecb_sentiment[["date", "ecb_count"]]
sentiment = pd.merge(
    sentiment, ecb_sentiment, how="left", left_on="date", right_on="date"
)
sentiment["ecb_count"] = sentiment["ecb_count"].ffill()

ecb_sentiment = sentiment.copy(deep=True)
ecb_sentiment = ecb_sentiment[["ecb_count", "stoxx"]]
ecb_sentiment["log_returns_eu"] = np.log(ecb_sentiment["stoxx"] + 1)
ecb_sentiment = ecb_sentiment[["ecb_count", "log_returns_eu"]]
ecb_sentiment = ecb_sentiment.groupby("ecb_count").sum().reset_index()
ecb_sentiment = ecb_sentiment.groupby(ecb_sentiment["ecb_count"]).mean().reset_index()

ecb_temp = sentiment.copy(deep=True)
ecb_temp = ecb_temp[["ecb_count", "press_conferences"]]
ecb_temp = ecb_temp.dropna(subset=["press_conferences"])

ecb_sentiment = pd.merge(
    ecb_sentiment, ecb_temp, how="left", left_on="ecb_count", right_on="ecb_count"
)

In [9]:
# Create lagged columns
for column in sentiment.columns:
    if column != "date":  # Skip the 'date' column
        sentiment[f"{column}_1"] = sentiment[column].shift(1)

for i in range(1, 5):
    sentiment[f"fed_minutes_forward_{i}"] = sentiment["fed_minutes"].shift(-i)
    sentiment[f"press_conferences_forward_{i}"] = sentiment["press_conferences"].shift(
        -i
    )
for i in range(0, 6):
    fed_sentiment[f"log_returns_{i}"] = fed_sentiment["log_returns"].shift(i)
    ecb_sentiment[f"log_returns_eu_{i}"] = ecb_sentiment["log_returns_eu"].shift(i)

# The count runs from day of announcment till day before next announcment so we shift the values by 1 so fed_minutes is now t where as before it was t-1
fed_sentiment["fed_minutes_t"] = fed_sentiment["fed_minutes"].shift(-1)
ecb_sentiment["press_conferences_t"] = ecb_sentiment["press_conferences"].shift(-1)

In [10]:
fed_sentiment = fed_sentiment.dropna()
ecb_sentiment = ecb_sentiment.dropna()

fed_sentiment = fed_sentiment[fed_sentiment["log_returns_5"] != 0]
ecb_sentiment = ecb_sentiment[ecb_sentiment["log_returns_eu_5"] != 0]

In [11]:
sentiment = sentiment[
    (sentiment["date"] >= "2000-01-01") & (sentiment["date"] <= "2022-06-30")
]

In [12]:
x0 = [
    "fed_minutes",
    "log_returns_0",
    "log_returns_1",
    "log_returns_2",
    "log_returns_3",
    "log_returns_4",
    "log_returns_5",
    # "log_returns_eu_6",
    # "log_returns_eu_7",
]

x1 = [
    "press_conferences",
    "log_returns_eu_0",
    "log_returns_eu_1",
    "log_returns_eu_2",
    "log_returns_eu_3",
    "log_returns_eu_4",
    "log_returns_eu_5",
]

# x2 = [
#     "minute_sentiment_1",
#     "log_returns_eu_0",
#     "log_returns_eu_2",
#     "log_returns_eu_4",
#     "log_returns_eu_6",
# ]

# x3 = [
#     "pc_sentiment_1",
#     "log_returnse_us_0",
#     "log_returnse_us_2",
#     "log_returnse_us_4",
#     "log_returnse_us_6",
# ]

yvar = "fed_minutes_t"
yvar1 = "press_conferences_t"
# yvar2 = "minute_sentiment"
# yvar3 = "pc_sentiment"

exog0 = (sm.add_constant(fed_sentiment[x0])).dropna()
exog1 = (sm.add_constant(ecb_sentiment[x1])).dropna()
# exog2 = (sm.add_constant(sentiment_mb[x2])).dropna()
# exog3 = (sm.add_constant(sentiment_mbe[x3])).dropna()

reg0 = sm.OLS(endog=fed_sentiment[yvar].loc[exog0.index], exog=exog0).fit(
    cov_type="HC0"
)
reg1 = sm.OLS(endog=ecb_sentiment[yvar1].loc[exog1.index], exog=exog1).fit(
    cov_type="HC0"
)
# reg2 = sm.OLS(endog=sentiment_mb[yvar2].loc[exog2.index], exog=exog2).fit(
#     cov_type="HC0"
# )
# reg3 = sm.OLS(endog=sentiment_mbe[yvar3].loc[exog3.index], exog=exog3).fit(
#     cov_type="HC0"
# )

stargazer = Stargazer([reg0, reg1])
stargazer.title(
    "Regressing Fed Sentiment on SP500 Returns and Regressing ECB Sentiment on STOXX 600 Returns"
)
stargazer.custom_columns(["Fed(t)", "ECB(t)"])
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)
stargazer.covariate_order(
    [
        "log_returns_0",
        "log_returns_1",
        "log_returns_2",
        "log_returns_3",
        "log_returns_4",
        "log_returns_5",
        "log_returns_eu_0",
        "log_returns_eu_1",
        "log_returns_eu_2",
        "log_returns_eu_3",
        "log_returns_eu_4",
        "log_returns_eu_5",
        "fed_minutes",
        "press_conferences",
    ]
)
stargazer.rename_covariates(
    {
        "log_returns_0": "SP00 Returns(t)",
        "log_returns_1": "SP00 Returns(t-1)",
        "log_returns_2": "SP00 Returns(t-2)",
        "log_returns_3": "SP00 Returns(t-3)",
        "log_returns_4": "SP00 Returns(t-4)",
        "log_returns_5": "SP00 Returns(t-5)",
        "fed_minutes": "Fed(t-1)",
        "log_returns_eu_0": "STOXX600 Returns(t)",
        "log_returns_eu_1": "STOXX600 Returns(t-1)",
        "log_returns_eu_2": "STOXX600 Returns(t-2)",
        "log_returns_eu_3": "STOXX600 Returns(t-3)",
        "log_returns_eu_4": "STOXX600 Returns(t-4)",
        "log_returns_eu_5": "STOXX600 Returns(t-5)",
        "press_conferences": "ECB(t-1)",
    },
)
stargazer.add_custom_notes(
    [
        "The unit for t is the time between a given meeting and the previous meeting for the given Central Bank."
    ]
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

\begin{table}[H] \centering
  \caption{Regressing Fed Sentiment on SP500 Returns and Regressing ECB Sentiment on STOXX 600 Returns}
\begin{tabular}{@{\extracolsep{5pt}}lcc}
\\hline
\hline \
\ & \multicolumn{1}{c}{Fed(t)} & \multicolumn{1}{c}{ECB(t)}  \\
\hline \
 SP00 Returns(t) & 0.068$^{}$ & \\
& (0.056) & \\
 SP00 Returns(t-1) & 0.290$^{***}$ & \\
& (0.056) & \\
 SP00 Returns(t-2) & 0.122$^{**}$ & \\
& (0.051) & \\
 SP00 Returns(t-3) & 0.200$^{***}$ & \\
& (0.051) & \\
 SP00 Returns(t-4) & 0.102$^{**}$ & \\
& (0.049) & \\
 SP00 Returns(t-5) & 0.126$^{**}$ & \\
& (0.054) & \\
 STOXX600 Returns(t) & & 0.079$^{}$ \\
& & (0.079) \\
 STOXX600 Returns(t-1) & & 0.130$^{**}$ \\
& & (0.062) \\
 STOXX600 Returns(t-2) & & 0.190$^{***}$ \\
& & (0.065) \\
 STOXX600 Returns(t-3) & & 0.036$^{}$ \\
& & (0.062) \\
 STOXX600 Returns(t-4) & & 0.079$^{}$ \\
& & (0.068) \\
 STOXX600 Returns(t-5) & & 0.012$^{}$ \\
& & (0.059) \\
 Fed(t-1) & 0.376$^{***}$ & \\
& (0.076) & \\
 ECB(t-1) & & 0.476$^{***}$ \\

In [ ]:
# New Market Analysis

x0 = [
    "fed_minutes",
    "log_returns_0",
    "log_returns_1",
    "log_returns_2",
    "log_returns_3",
    "log_returns_4",
    "log_returns_5",
    # "log_returns_eu_6",
    # "log_returns_eu_7",
]

x1 = [
    "press_conferences",
    "log_returns_eu_0",
    "log_returns_eu_1",
    "log_returns_eu_2",
    "log_returns_eu_3",
    "log_returns_eu_4",
    "log_returns_eu_5",
]

# x2 = [
#     "minute_sentiment_1",
#     "log_returns_eu_0",
#     "log_returns_eu_2",
#     "log_returns_eu_4",
#     "log_returns_eu_6",
# ]

# x3 = [
#     "pc_sentiment_1",
#     "log_returnse_us_0",
#     "log_returnse_us_2",
#     "log_returnse_us_4",
#     "log_returnse_us_6",
# ]

yvar = "fed_minutes_t"
yvar1 = "press_conferences_t"
# yvar2 = "minute_sentiment"
# yvar3 = "pc_sentiment"

exog0 = (sm.add_constant(fed_sentiment[x0])).dropna()
exog1 = (sm.add_constant(ecb_sentiment[x1])).dropna()
# exog2 = (sm.add_constant(sentiment_mb[x2])).dropna()
# exog3 = (sm.add_constant(sentiment_mbe[x3])).dropna()

reg0 = sm.OLS(endog=fed_sentiment[yvar].loc[exog0.index], exog=exog0).fit(
    cov_type="HC0"
)
reg1 = sm.OLS(endog=ecb_sentiment[yvar1].loc[exog1.index], exog=exog1).fit(
    cov_type="HC0"
)
# reg2 = sm.OLS(endog=sentiment_mb[yvar2].loc[exog2.index], exog=exog2).fit(
#     cov_type="HC0"
# )
# reg3 = sm.OLS(endog=sentiment_mbe[yvar3].loc[exog3.index], exog=exog3).fit(
#     cov_type="HC0"
# )

stargazer = Stargazer([reg0, reg1])
stargazer.title(
    "Regressing Fed Sentiment on SP500 Returns and Regressing ECB Sentiment on STOXX 600 Returns"
)
stargazer.custom_columns(["Fed(t)", "ECB(t)"])
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)
stargazer.covariate_order(
    [
        "log_returns_0",
        "log_returns_1",
        "log_returns_2",
        "log_returns_3",
        "log_returns_4",
        "log_returns_5",
        "log_returns_eu_0",
        "log_returns_eu_1",
        "log_returns_eu_2",
        "log_returns_eu_3",
        "log_returns_eu_4",
        "log_returns_eu_5",
        "fed_minutes",
        "press_conferences",
    ]
)
stargazer.rename_covariates(
    {
        "log_returns_0": "SP00 Returns(t)",
        "log_returns_1": "SP00 Returns(t-1)",
        "log_returns_2": "SP00 Returns(t-2)",
        "log_returns_3": "SP00 Returns(t-3)",
        "log_returns_4": "SP00 Returns(t-4)",
        "log_returns_5": "SP00 Returns(t-5)",
        "fed_minutes": "Fed(t-1)",
        "log_returns_eu_0": "STOXX600 Returns(t)",
        "log_returns_eu_1": "STOXX600 Returns(t-1)",
        "log_returns_eu_2": "STOXX600 Returns(t-2)",
        "log_returns_eu_3": "STOXX600 Returns(t-3)",
        "log_returns_eu_4": "STOXX600 Returns(t-4)",
        "log_returns_eu_5": "STOXX600 Returns(t-5)",
        "press_conferences": "ECB(t-1)",
    },
)
stargazer.add_custom_notes(
    [
        "The unit for t is the time between a given meeting and the previous meeting for the given Central Bank."
    ]
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)